<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 15:41:47
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 23.19 K (0.17%)
Current PnL: -13.38 L (-9.32%)
CY Booked + Current PnL: -6.42 L (-4.47%)
-------------------
Total profit:  4.46 L
Total loss:  -17.84 L
-------------------
Total Booked + Current PnL: 20.56 L (17.64%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.82 L (52.53%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-19.08,H-LC,84.38,111912.0,7600.0,9938.0,0.95,7.29,8.88,16.81,27.0,0.76,0.82,24.50,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-16.85,H-LC,87.50,113302.0,9736.0,10968.0,-0.15,9.40,9.68,20.00,32.0,0.89,0.83,24.80,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,0.15,M-LC,28.12,174063.0,33223.0,11262.0,0.99,23.59,6.47,31.59,88.0,2.95,1.27,51.20,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,50.53,M-SC,94.79,131112.0,-11367.0,11984.0,-1.14,-7.98,9.14,0.44,247.0,-0.95,0.96,54.75,OX40N,NTT,BANKS
71,SBIN,760.30,863.00,-11.51,M-LC,34.90,215140.0,13660.0,13554.0,0.59,6.78,6.30,13.51,77.0,1.01,1.57,20.23,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,19.79,181192.0,1384.0,111034.0,-0.57,0.77,61.28,62.52,173.0,0.01,1.33,35.12,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,57.29,152075.0,1162.0,59294.0,0.80,0.77,38.99,40.06,103.0,0.02,1.11,24.39,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,RELAXO,902.64,1176.00,-34.25,H-SC,72.92,59488.0,-56050.0,91040.0,6.20,-48.51,153.04,30.28,131.0,-0.62,0.44,16.22,X40N,NTT,FOOTWEAR
28,DIGITIDE,188.38,237.72,NaN,NaN,10.42,48995.0,7740.0,3067.0,4.77,18.76,6.26,26.19,272.0,2.52,0.36,15.35,XY24,ATH,IT
33,GREENPANEL,375.16,537.00,217.83,M-SC,88.54,141788.0,-41290.0,120265.0,4.50,-22.55,84.82,43.14,217.0,-0.34,1.04,33.37,XY24,NTT,MISC
70,SATIN,199.77,274.00,-14.59,H-SC,89.58,169389.0,-26585.0,99397.0,3.72,-13.57,58.68,37.16,135.0,-0.27,1.24,26.50,XY24,NTT,FINANCE
55,LALPATHLAB,2804.19,3545.00,-31.89,H-SC,30.21,156695.0,2465.0,38281.0,3.35,1.60,24.43,26.42,127.0,0.06,1.15,23.57,X40N,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-163.64,M-SC,27.08,73914.0,-28296.0,121685.0,-9.24,-27.68,164.63,91.37,212.0,-0.23,0.54,19.99,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-29.71,H-SC,71.88,208777.0,-39753.0,255230.0,-2.15,-16.00,122.25,86.70,134.0,-0.16,1.53,38.12,XY24,NTT,BANKS
30,DMART,3764.92,5201.00,-16.87,X-LC,61.46,208794.0,24313.0,46060.0,-2.01,13.18,22.06,38.14,15.0,0.53,1.53,25.18,X40N,NTT,FMCG
3,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98694.0,-2269.0,96799.0,-1.96,-2.25,98.08,93.63,67.0,-0.02,0.72,47.00,SR,ATH,CHEMICALS
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,85.42,202522.0,30901.0,131173.0,-1.71,18.01,64.77,94.43,201.0,0.24,1.48,49.47,XR,NTT,DURABLES


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,50.53,M-SC,94.79,131112.0,-11367.0,11984.0,-1.14,-7.98,9.14,0.44,247.0,-0.95,0.96,54.75,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.51,M-SC,11.46,148122.0,-23019.0,22959.0,0.13,-13.45,15.50,-0.03,191.0,-1.00,1.08,6.46,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.18,H-SC,96.88,81608.0,-13708.0,13751.0,1.32,-14.38,16.85,0.04,157.0,-1.00,0.60,60.96,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,126.56,H-SC,91.67,148738.0,-25821.0,25851.0,-0.15,-14.79,17.38,0.02,133.0,-1.00,1.09,28.98,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.14,X-MC,16.67,86926.0,-15726.0,15725.0,-0.10,-15.32,18.09,-0.00,59.0,-1.00,0.64,32.14,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.06,X-LC,39.58,266744.0,-29190.0,123102.0,0.56,-9.86,46.15,31.73,1.0,-0.24,1.95,6.77,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.47,X-LC,67.71,293685.0,32084.0,113539.0,1.36,12.26,38.66,55.67,2.0,0.28,2.15,19.12,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.38,X-LC,29.17,262002.0,-8345.0,42968.0,1.12,-3.09,16.40,12.81,5.0,-0.19,1.92,9.57,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,15.77,X-LC,59.38,265489.0,36256.0,31036.0,-0.40,15.82,11.69,29.35,10.0,1.17,1.94,26.15,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.64,X-LC,26.04,82310.0,-27862.0,116938.0,-0.58,-25.29,142.07,80.85,11.0,-0.24,0.60,21.23,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.9,NaN,NaN,68.75,72350.0,37901.0,2048.0,-0.97,110.02,2.83,115.97,274.0,18.51,0.53,10.64,AR,ATH,ELECTRICAL
88,UNIONBANK,123.87,163.0,0.15,M-LC,28.12,174063.0,33223.0,11262.0,0.99,23.59,6.47,31.59,88.0,2.95,1.27,51.20,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,50.00,231988.0,7026.0,160628.0,-1.30,3.12,69.24,74.53,189.0,0.04,1.70,34.69,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,27.50,H-SC,52.08,130372.0,5075.0,54039.0,0.80,4.05,41.45,47.18,143.0,0.09,0.95,63.10,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.43,H-SC,43.75,118490.0,16486.0,28532.0,-0.62,16.16,24.08,44.14,126.0,0.58,0.87,35.03,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,28.12,174063.0,33223.0,11262.0,0.99,23.59,6.47,31.59,88.0,2.95,1.27,51.20,XY24,NTT,BANKS
15,BLUESTARCO,1646.70,2337.55,-1.90,H-SC,97.92,184120.0,19450.0,49639.0,-0.06,11.81,26.96,41.95,124.0,0.39,1.35,20.16,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,68.75,72350.0,37901.0,2048.0,-0.97,110.02,2.83,115.97,274.0,18.51,0.53,10.64,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,85.42,202522.0,30901.0,131173.0,-1.71,18.01,64.77,94.43,201.0,0.24,1.48,49.47,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,83.33,167431.0,16486.0,92974.0,1.12,10.92,55.53,72.51,71.0,0.18,1.23,15.33,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.08,H-SC,79.17,134975.0,11639.0,122125.0,-1.29,9.44,90.48,108.45,123.0,0.10,0.99,33.90,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,27.50,H-SC,52.08,130372.0,5075.0,54039.0,0.80,4.05,41.45,47.18,143.0,0.09,0.95,63.10,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,4.17,17470.0,-32123.0,1046.0,1.26,-64.77,5.99,-62.66,270.0,-30.71,0.13,17.15,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67469.0,-46080.0,86131.0,0.17,-40.58,127.66,35.27,265.0,-0.53,0.49,65.13,XR,NTT,HOTELS
58,MASFIN,326.60,399.50,-18.25,H-SC,78.12,93075.0,-4905.0,26778.0,0.93,-5.01,28.77,22.32,145.0,-0.18,0.68,34.99,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-2.07,X-LC,47.92,97726.0,-19.0,37419.0,0.69,-0.02,38.29,38.26,18.0,-0.00,0.72,30.49,X40,BTT,FINANCE
3,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98694.0,-2269.0,96799.0,-1.96,-2.25,98.08,93.63,67.0,-0.02,0.72,47.00,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-28.73,L-SC,100.00,143508.0,1666.0,54906.0,1.50,1.17,38.26,39.88,262.0,0.03,1.05,45.43,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98694.0,-2269.0,96799.0,-1.96,-2.25,98.08,93.63,67.0,-0.02,0.72,47.00,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-1.90,H-SC,97.92,184120.0,19450.0,49639.0,-0.06,11.81,26.96,41.95,124.0,0.39,1.35,20.16,X40N,ATH,AC
92,VIPIND,488.80,489.00,-213.18,H-SC,96.88,81608.0,-13708.0,13751.0,1.32,-14.38,16.85,0.04,157.0,-1.00,0.60,60.96,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67469.0,-46080.0,86131.0,0.17,-40.58,127.66,35.27,265.0,-0.53,0.49,65.13,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.91
1,25,41.87
2,50,70.49


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.48
LC    33.93
MC    21.60
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.79
X40      13.00
X40N     11.97
XY25     11.97
XR       10.13
OX40N     8.21
AR        8.18
MH        1.70
X5K       1.47
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.69
M-SC    15.94
H-LC    14.12
X-LC    12.86
H-MC     9.54
X-MC     7.62
M-LC     5.90
X-SC     4.40
M-MC     4.08
L-SC     2.45
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-1.72,35.76
IT,12.15,-9.46,63.22
FINANCE,8.56,-11.39,51.81
BANKS,7.84,-6.51,49.97
MISC,5.91,-19.38,74.94
PAINTS,5.72,-11.92,33.97
ELECTRICAL,5.50,2.06,43.68
HEALTHCARE,4.59,-2.70,32.70
AUTO,4.34,-13.95,64.05


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2682825.0,26
XR,1075754.0,13
AR,1023768.0,10
X40,679700.0,10
X40N,530569.0,11
XY25,484382.0,8
OX40N,308292.0,11
SR,177984.0,2
MH,83370.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1959343.0,20
M-SC,1760979.0,21
H-LC,610154.0,12
X-LC,553448.0,8
H-MC,531974.0,8
M-MC,413357.0,3
X-MC,412438.0,6
X-SC,317759.0,5
L-SC,262214.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      872622.0      6
M-SC       XY24      806896.0      7
H-SC       AR        523219.0      3
M-SC       XR        335167.0      4
M-MC       XY24      302323.0      2
H-MC       XY24      298908.0      3
X-LC       X40       274060.0      3
H-SC       XR        212249.0      3
           X40N      207492.0      4
H-LC       X40N      194985.0      4
M-SC       AR        181914.0      2
           OX40N     162981.0      5
H-LC       X40       155029.0      3
X-MC       X40       148036.0      2
L-SC       XR        147769.0      2
M-LC       XY24      147730.0      2
X-SC       XR        122767.0      1
M-SC       XY25      121685.0      1
X-LC       AR        116938.0      1
           XY25      116390.0      3
M-MC       XR        111034.0      1
X-MC       XY25      102998.0      1
X-SC       SR         96799.0      1
X-MC       XY24       93552.0      1
M-LC       XR         92974.0      1
H-SC       OX40N      89722.0      3
H-MC       XY25       83112.0      1
M-SC       SR         81185.0      1
H-LC       AR         80816.0      1
M-SC       X40        71151.0      1
H-LC       X200       68914.0      1
           X5K        66964.0      2
L-SC       AR         59539.0      1
H-MC       AR         59294.0      1
X-SC       XY24       58329.0      1
L-SC       XY24       54906.0      1
H-SC       MH         54039.0      1
L-MC       XR         53794.0      1
X-MC       X40N       52127.0      1
L-LC       XY25       46643.0      1
X-LC       X40N       46060.0      1
H-LC       XY24       43446.0      1
X-SC       OX40N      39864.0      2
H-MC       X40        31424.0      1
           X40N       29905.0      1
           MH         29331.0      1
X-MC       OX40N      15725.0      1
M-LC       XY25       13554.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
